In [5]:
from io import open
import glob
import os
import unicodedata
import string
import torch
import torch.nn as nn
import random
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import codecs
import numpy as np

In [6]:
# helper functions
def findFiles(path):
    """
    Find files under the given path.
    """
    return glob.glob(path)

def unicodeToAscii(s, vocab=string.ascii_letters+" .,;'"):
    """
    Turn a unicode string to plain ASCII.
    Params:
        s: string to be converted
        vocab: vocabulary as in unique characters, default to all ascii chars.
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in vocab
    )

In [7]:
# IO
data = codecs.open('../data/potter.txt', 'r', encoding='utf8', errors='ignore').read()
predict = codecs.open('../data/output.txt', 'w', encoding='utf8')
chars = list(set(data))
data_size = len(data) 
vocab_size = len(chars)

print(f'data has {data_size} characters,{vocab_size} unique.')              # data has 1109177 characters,80 unique.

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

print(char_to_ix)
print(ix_to_char)


data has 1109177 characters,80 unique.
{'R': 0, 'L': 1, 'h': 2, '7': 3, '0': 4, 'b': 5, 'W': 6, 'B': 7, 'g': 8, '2': 9, '\n': 10, 'l': 11, 'J': 12, '6': 13, 'c': 14, ' ': 15, '}': 16, '-': 17, 'I': 18, ')': 19, 'z': 20, 'T': 21, 'e': 22, '?': 23, ':': 24, '\t': 25, '!': 26, 'd': 27, 'q': 28, 'u': 29, 'N': 30, '"': 31, 'G': 32, '/': 33, 'S': 34, 'A': 35, ';': 36, 'X': 37, 'k': 38, 'D': 39, 'x': 40, ',': 41, 'Y': 42, 'Z': 43, 'w': 44, 'Q': 45, 'P': 46, 'U': 47, 'a': 48, 'H': 49, '_': 50, '8': 51, 'M': 52, 'm': 53, 's': 54, 'O': 55, '9': 56, 'y': 57, 'i': 58, '3': 59, 'K': 60, 'n': 61, 'p': 62, 'j': 63, 'o': 64, 'V': 65, 'f': 66, '4': 67, 'F': 68, 'r': 69, "'": 70, '1': 71, '(': 72, 'E': 73, 't': 74, '.': 75, 'v': 76, '5': 77, '^': 78, 'C': 79}
{0: 'R', 1: 'L', 2: 'h', 3: '7', 4: '0', 5: 'b', 6: 'W', 7: 'B', 8: 'g', 9: '2', 10: '\n', 11: 'l', 12: 'J', 13: '6', 14: 'c', 15: ' ', 16: '}', 17: '-', 18: 'I', 19: ')', 20: 'z', 21: 'T', 22: 'e', 23: '?', 24: ':', 25: '\t', 26: '!', 27: 'd', 28:

In [10]:
# hyperparameters
hidden_size = 256          # size of hidden layer of neurons
seq_length = 128           # number of steps to unroll the RNN for
learning_rate = 1e-1


# model parameters
W_xh = torch.randn((hidden_size, vocab_size), requires_grad=True) * 0.01        # weight: input to hidden
W_hh = torch.randn(hidden_size, hidden_size, requires_grad=True) * 0.01         # weight: hidden to hidden, notice shape () is omitable
W_hy = torch.randn((vocab_size, hidden_size), requires_grad=True) * 0.01        # weight: hidden to output
b_h = torch.zeros((hidden_size, 1))                          # hidden bias
b_y = torch.zeros((vocab_size, 1))                           # output bias

In [14]:
W_xh

tensor([[-0.0019,  0.0017,  0.0044,  ...,  0.0145,  0.0024, -0.0053],
        [ 0.0154, -0.0002, -0.0034,  ..., -0.0107,  0.0005, -0.0073],
        [ 0.0061, -0.0102,  0.0010,  ..., -0.0098,  0.0107,  0.0051],
        ...,
        [-0.0096, -0.0042, -0.0127,  ..., -0.0002,  0.0093, -0.0048],
        [-0.0172,  0.0064, -0.0142,  ..., -0.0070, -0.0138, -0.0123],
        [-0.0010, -0.0053, -0.0154,  ..., -0.0102, -0.0125,  0.0027]],
       grad_fn=<MulBackward>)

In [15]:
W_hh

tensor([[ 0.0156,  0.0050,  0.0032,  ...,  0.0063,  0.0030,  0.0075],
        [ 0.0003,  0.0083, -0.0093,  ..., -0.0079,  0.0092,  0.0037],
        [-0.0024, -0.0006,  0.0108,  ...,  0.0081, -0.0055,  0.0166],
        ...,
        [ 0.0096,  0.0102,  0.0066,  ...,  0.0172, -0.0019, -0.0017],
        [ 0.0101,  0.0013,  0.0171,  ...,  0.0176,  0.0015, -0.0088],
        [ 0.0063, -0.0052,  0.0205,  ..., -0.0043, -0.0033, -0.0081]],
       grad_fn=<MulBackward>)

In [16]:
W_hy

tensor([[-0.0021, -0.0024,  0.0006,  ...,  0.0009,  0.0097, -0.0074],
        [ 0.0179,  0.0094, -0.0043,  ...,  0.0006, -0.0180, -0.0037],
        [ 0.0024, -0.0092,  0.0072,  ...,  0.0006, -0.0011, -0.0085],
        ...,
        [-0.0060,  0.0025, -0.0125,  ...,  0.0090,  0.0108, -0.0109],
        [-0.0109, -0.0011,  0.0039,  ..., -0.0007,  0.0031, -0.0103],
        [-0.0046, -0.0138, -0.0120,  ...,  0.0026, -0.0021, -0.0006]],
       grad_fn=<MulBackward>)

In [23]:
b_h.transpose_(0,1).shape

torch.Size([1, 256])

In [24]:
m = torch.tensor([1,2,3,4])

In [25]:
m

tensor([1, 2, 3, 4])

In [26]:
n = np.copy(m)

In [41]:
n
n.shape

torch.Size([4])

In [28]:
n = torch.tensor(m)

In [29]:
n

tensor([1, 2, 3, 4])

In [30]:
m *= 2

In [31]:
m

tensor([2, 4, 6, 8])

In [32]:
n

tensor([1, 2, 3, 4])

In [43]:
torch.tensor([2,3]).unsqueeze_(0).transpose_(0,1)

tensor([[2],
        [3]])